## Distillation

A destilação de modelo, também conhecida como destilação de conhecimento, é uma técnica usada no aprendizado profundo para transferir conhecimento de um modelo grande e complexo (muitas vezes chamado de modelo "professor") para um modelo menor e mais eficiente (o modelo "aluno") . O objetivo da destilação do modelo é permitir que o modelo do aluno alcance um desempenho próximo ao do modelo do professor, ao mesmo tempo que é mais eficiente computacionalmente e requer menos recursos para ser executado.

Processo:
- Preparação: Treine o modelo do professor na tarefa alvo até atingir alto desempenho. O modelo do aluno é então inicializado, potencialmente com uma arquitetura diferente e mais leve.
- Escala de temperatura: Aplique a escala de temperatura às saídas dos modelos do professor e do aluno para suavizar suas distribuições de saída.
- Treinando o Aluno: Treine o modelo do aluno usando uma perda de destilação que o incentiva a imitar a distribuição suavizada de produção do professor. O modelo de estudante também pode ser treinado usando a perda tradicional do rótulo rígido, caso em que a perda total seria uma soma ponderada da perda de destilação e da perda do rótulo rígido.
- Ajuste fino: Opcionalmente, o modelo do aluno pode ser ajustado ainda mais na tarefa alvo usando os rótulos rígidos originais, melhorando seu desempenho.

In [2]:
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np

# Load a smaller pre-trained model as the "teacher"
teacher_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load a subset of the SST-2 dataset for demonstration
dataset = load_dataset('glue', 'sst2', split='train[:10%]')

# Preprocess the data
def preprocess_function(examples):
    return tokenizer(examples['sentence'], padding=True, truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

class StudentModel(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_size, num_classes, vocab_size):
        super(StudentModel, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)  # Embedding layer
        self.fc1 = torch.nn.Linear(embedding_dim, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
    
    def forward(self, input_ids):
        # Convert input_ids to embeddings
        embeddings = self.embedding(input_ids)  # Shape: [batch_size, seq_length, embedding_dim]

        # Pool the embeddings along the sequence length dimension
        pooled_output = torch.mean(embeddings, dim=1)  # Shape: [batch_size, embedding_dim]

        hidden = self.fc1(pooled_output)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        return output

# Initialize the student model with the correct dimensions
vocab_size = tokenizer.vocab_size  # Assuming you're using the DistilBert tokenizer
embedding_dim = teacher_model.config.dim  # DistilBert's embedding dimension
student_model = StudentModel(embedding_dim=embedding_dim, hidden_size=256, num_classes=2, vocab_size=vocab_size)

# Prepare for training
train_dataset = tokenized_dataset.remove_columns(['sentence', 'idx'])
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
train_loader = DataLoader(train_dataset, batch_size=16)

# Define loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(student_model.parameters(), lr=1e-4)

# Distillation training loop
num_epochs = 3
for epoch in range(num_epochs):
    student_model.train()
    for batch in train_loader:
        # Forward pass of the teacher model with input_ids
        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=batch['input_ids']).logits
        
        # Forward pass of the student model without converting input_ids to float
        student_logits = student_model(batch['input_ids'])
        
        # Compute distillation loss
        loss = loss_fn(student_logits, teacher_logits.argmax(dim=1))
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# The student model is now distilled and can be used for inference or further training

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

c:\Users\luiza\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\luiza\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/6735 [00:00<?, ? examples/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


RuntimeError: stack expects each tensor to be equal size, but got [56] at entry 0 and [53] at entry 8